Department of Physics, University of Pisa (AA 2023-2024)

### **Multimessenger Physics Laboratory tutorial series**




## **Tutorial on Object-oriented Python programming. An astronomical example**
#### (M. Razzano, Feb 26, 2024)

In this tutorial we will see how to write classes in Python and use this object-oriented programming 
to solve an astronomical problem. 
The goal is to build a system to store and query a catalog of stars with exoplanets and determine their observability from a certain observatory. 
We will also use some function of astropy

In [1]:
#First, import basic modules
import copy

import numpy as np
#import Numpy
import os
import numpy as np
import pandas as pd

# Import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

#Import astropy
from astropy.coordinates import EarthLocation
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.coordinates import AltAz

#Setup the main directories using the environ dir
#Define the various directories
tutorials_dir = os.getcwd()

#using dirname you can go up of one directory
main_dir = os.path.dirname(tutorials_dir)

#then use join to define variables pointing to subdirectories
data_dir = os.path.join(main_dir,"data")

print("Project main dir is %s" % main_dir)
print("Data dir is %s" %data_dir)
print("Tutorials dir is %s" %tutorials_dir)

Project main dir is /home/jovyan/experience-grbs-template
Data dir is /home/jovyan/experience-grbs-template/data
Tutorials dir is /home/jovyan/experience-grbs-template/tutorials


<h2>Define the basic classes</h2>
We will first define a class describing an astronomical source, which will be fairly general. 
Then we will define an exoplanet class, that is inheriting from the previous class but that contains more information

In [2]:
#First, we define a generic class for an astronomical source

#Class names are usually Uppercase
class AstroSource(object):    
    "Common base class for all astronomical sources"

    #This is the constructor, where the data members are declared and initialized
    def __init__(self, m_name):
        "description for the constructor"
        #define the members
        self.__name = m_name
        self.__type = None
        
        #coordinates
        self.__ra_deg = None
        self.__dec_deg = None
        self.__distance = None
        self.__distance_unit = None
    
    #here we can put some functions that set the members. Here we put just some examples, you should fill
    #with all the relevant data members
    def get_coordinates_decimal(self):
        return self.__ra_deg, self.__dec_deg
    
    def set_coordinates_decimal(self, m_coord1, m_coord2, m_system="icrs"):
        if m_system=="icrs":
            self.__ra_deg = m_coord1
            self.__dec_deg = m_coord2
    
    def set_distance(self, m_distance, m_distance_unit):
        self.__distance = m_distance
        self.__distance_unit = m_distance_unit

    def set_equatorial_coordinates(self, m_ra, m_dec):        
       self.__ra_deg = m_ra
       self.__dec_deg = m_dec


    def set_name(self, m_name):        
        self.__name = m_name

    def set_type(self, m_type):        
        self.__type = m_type
        
    def twinkle(self):
        "just twinkle"
        print("Hi, I'm the %s %s and I am twinkling !" % (self.__type, self.__name))

    #Here some functions to get the variables    
    def get_name(self):
        return self.__name

    def get_equatorial_coordinates(self):
        return self.__ra, self.__dec

    #and a function that prints the summary of the source
    def print_summary(self):
        print("** Summary for source %s" % self.__name)
        print("   Type: %s" % self.__type)
        print("   Distance: %3.f %s" % (self.__distance, self.__distance_unit))

In [3]:
#Then, we define an exoplanet class, that inherit from AstroSource
class Exoplanet(AstroSource):    
    #Here you can put a short description
    "Class for exoplanet"

    def __init__(self,m_name):        
        super(Exoplanet,self).__init__(m_name)
        self.set_type("planet")
        self.__host_star_name = None
            
    def set_host_star_name(self,m_star_name):
        "star name"
        self.__host_star_name(m_star_name)

In [4]:
#Where is a star? Where does it rise, or culminate from a certain point on Earth?
#To compute this, we can create a Observatory class

class Observatory(object):
    "observatory class. this relies on astropy"
    
    def __init__(self,m_name):
        "constructor"
        self.__name = m_name
        self.__latitude = None
        self.__longitude = None
        self.__running_time = None
        self.__latitude_string=None
        self.__longitude_string=None
        self.__altitude_meters=None
        self.__location=None
        self.__time=None
        
    def get_altaz_coordinates_from_radec_decimal(self, m_ra, m_dec):
        "return alt and az of a point in the sky "
        aa = AltAz(location=self.__location, obstime=self.__time) 
        m_sky_coords = SkyCoord(m_ra*u.deg, m_dec*u.deg, frame='icrs')
        m_altaz = m_sky_coords.transform_to(aa)
        
        return m_altaz.alt.deg,m_altaz.az.deg

    def get_time(self):
        return self.__time
    
    def set_location(self, m_latitude, m_longitude, m_altitude):
        "coordinates"
        #String of latitude is in format: 31d57.5m
        self.__latitude_string = m_latitude
        self.__longitude_string = m_longitude
        self.__altitude_meters = m_altitude
        self.__location = EarthLocation(lat=self.__latitude_string, lon=self.__longitude_string, height=self.__altitude_meters*u.m)
        
    def set_time(self,m_time,m_scale="utc",m_format="isot"):
        self.__time = Time(m_time,format=m_format,scale=m_scale)

In [5]:
a_star = AstroSource("ProximaCen")
a_star.set_type("star")
a_star.set_distance(4.2, "ly")
print(a_star.get_name())

a_star.print_summary()

ProximaCen
** Summary for source ProximaCen
   Type: star
   Distance:   4 ly


In [6]:
a_planet = Exoplanet("CoRot-13b")
a_planet.set_distance(1060,"pc")
a_planet.print_summary()
a_planet.twinkle()

** Summary for source CoRot-13b
   Type: planet
   Distance: 1060 pc
Hi, I'm the planet CoRot-13b and I am twinkling !


<h2>Read the list of exoplanets and find those above the horizon</h2>

In [7]:
m_exo_filename = os.path.join(data_dir, "exoplanets_nasa_archive_20181129.csv")
exo_df = pd.read_csv(m_exo_filename, skiprows=146)
exo_df.sample(2)

,rowid,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_pnum,pl_orbper,pl_orbsmax,pl_orbeccen,pl_orbincl,...,st_bmvj,st_vjmic,st_vjmrc,st_jmh2,st_hmk2,st_jmk2,st_bmy,st_m1,st_c1,st_colorn
1149,1150,KELT-7,b,KELT-7 b,Transit,1,2.734770,NaN,0.0,83.76,...,0.430,NaN,NaN,0.159,0.037,0.196,NaN,NaN,NaN,6
597,598,HD 3167,b,HD 3167 b,Transit,3,0.959624,0.0186,NaN,NaN,...,0.826,NaN,NaN,0.345,0.137,0.482,NaN,NaN,NaN,6


In [8]:
# Read the file and fill the infos on the planets
exoplanet_list=[]
for pi in range(len(exo_df)):
    m_name = exo_df.iloc[pi]["pl_name"]
    new_exoplanet=Exoplanet(m_name)
    new_exoplanet.set_distance(exo_df.iloc[pi]["st_dist"],"pc")
    new_exoplanet.set_coordinates_decimal(exo_df.iloc[pi]["ra"],exo_df.iloc[pi]["dec"])
    exoplanet_list.append(new_exoplanet)
    
    
print("Loaded %d exoplanets" % len(exoplanet_list))

Loaded 3838 exoplanets


In [9]:
# Run over the first 10 and let them twinkle
for exoplanet in exoplanet_list[:10]:
    exoplanet.twinkle()

Hi, I'm the planet 11 Com b and I am twinkling !
Hi, I'm the planet 11 UMi b and I am twinkling !
Hi, I'm the planet 14 And b and I am twinkling !
Hi, I'm the planet 14 Her b and I am twinkling !
Hi, I'm the planet 16 Cyg B b and I am twinkling !
Hi, I'm the planet 18 Del b and I am twinkling !
Hi, I'm the planet 1RXS J160929.1-210524 b and I am twinkling !
Hi, I'm the planet 24 Boo b and I am twinkling !
Hi, I'm the planet 24 Sex b and I am twinkling !
Hi, I'm the planet 24 Sex c and I am twinkling !


In [ ]:
# Now, we can run over the exoplanet list and see if they are visible from a certain point of Earth

# Let's choose Pisa, and pick a time
obs_lat = "43d42.5118m"
obs_long = "10d24.216m"
obs_altitude = 0
obs_time = "2019-02-24T03:00:00.123456789"

pisa_obs = Observatory("Pisa Observatory")
pisa_obs.set_location(obs_lat, obs_long, obs_altitude)
pisa_obs.set_time(obs_time)

for pi in range(len(exo_df)):
    planet_ra, planet_dec = exoplanet_list[pi].get_coordinates_decimal()
    planet_alt,planet_az = pisa_obs.get_altaz_coordinates_from_radec_decimal(planet_ra,planet_dec)
    obs_status = "NOT VISIBLE"
    if planet_alt>0.:
        obs_status = "VISIBLE"

    print("Planet %s: (RA,DEC)=(%.3f,%.3f) --> alt=%.3f, az=%.3f --> status=%s" % (exoplanet_list[pi].get_name(),planet_ra,planet_dec,planet_alt,planet_az,obs_status))

Planet 11 Com b: (RA,DEC)=(185.179,17.793) --> alt=57.204, az=224.961 --> status=VISIBLE
Planet 11 UMi b: (RA,DEC)=(229.275,71.824) --> alt=60.311, az=12.591 --> status=VISIBLE
Planet 14 And b: (RA,DEC)=(352.823,39.236) --> alt=-0.797, az=27.084 --> status=NOT VISIBLE
Planet 14 Her b: (RA,DEC)=(242.601,43.818) --> alt=65.861, az=78.064 --> status=VISIBLE
Planet 16 Cyg B b: (RA,DEC)=(295.467,50.518) --> alt=34.184, az=50.040 --> status=VISIBLE
Planet 18 Del b: (RA,DEC)=(314.608,10.839) --> alt=-3.528, az=71.259 --> status=NOT VISIBLE
Planet 1RXS J160929.1-210524 b: (RA,DEC)=(242.376,-21.083) --> alt=18.232, az=147.133 --> status=VISIBLE
Planet 24 Boo b: (RA,DEC)=(217.158,49.845) --> alt=81.745, az=40.025 --> status=VISIBLE
Planet 24 Sex b: (RA,DEC)=(155.868,-0.902) --> alt=25.021, az=241.781 --> status=VISIBLE
Planet 24 Sex c: (RA,DEC)=(155.868,-0.902) --> alt=25.021, az=241.781 --> status=VISIBLE
Planet 2MASS J01225093-2439505 b: (RA,DEC)=(20.712,-24.664) --> alt=-69.726, az=21.918 -->